# 데이터 구조

In [ ]:
# 필요한 모듈 불러오기

import pandas as pd
import numpy as np
import itertools

In [ ]:
# 제공 데이터 불러오기

fw = pd.read_csv('../data/01_제공데이터_update(210811)/01_음식물쓰레기_FOOD_WASTE_210811_update.csv', encoding = 'cp949', parse_dates = ['base_date'])
korean = pd.read_csv('../data/01_제공데이터_update(210811)/02-1_내국인유동인구_KOREAN.csv', encoding = 'cp949', parse_dates = ['base_date'])
lt_frgn = pd.read_csv('../data/01_제공데이터_update(210811)/02-2_장기체류외국인유동인구_LONG_TERM_FRGN.csv', encoding = 'cp949', parse_dates = ['base_date'])
st_frgn = pd.read_csv('../data/01_제공데이터_update(210811)/02-3_단기체류외국인유동인구_SHORT_TERM_FRGN.csv', encoding = 'cp949', parse_dates = ['base_date'])
residence = pd.read_csv('../data/01_제공데이터_update(210811)/03_거주인구_RESIDENT_POP.CSV', encoding = 'cp949')
CS_home = pd.read_csv('../data/01_제공데이터_update(210811)/04_음식관련카드소비_CARD_SPENDING.csv', encoding = 'cp949', parse_dates = ['base_date'])

# 음식물쓰레기 데이터 가공

In [ ]:
fw['year'] = fw.base_date.dt.year
fw['month'] = fw.base_date.dt.month

In [ ]:
fw1 = fw.groupby(['year', 'month', 'emd_nm']).sum().reset_index()

In [ ]:
fw1['year'] = fw1['year'].astype(str)
fw1['month'] = fw1['month'].astype(str)
fw1['term'] = fw1['year'] + '-0' + fw1['month'] if len(fw1['month']) == 1 else fw1['year'] +'-'+ fw1['month']
fw1.drop(['year', 'month'], axis = 1, inplace = True)
fw1['term'] = fw1.term.apply(lambda x: x[:5] + '0' + x[-1] if len(x) == 6 else x) 

In [ ]:
fw1['term'] = fw1['term'].apply(lambda x : dt.datetime.strptime(x, '%Y-%m'))

In [ ]:
## 종속변수 데이터 저장
fw1[['term', 'emd_nm', 'em_g']].to_csv('../data/가공데이터/y_data.csv')

In [ ]:
fw1.drop(['pay_amt', 'em_cnt'], axis = 1, inplace = True)

In [ ]:
fw1['before_3'] = fw1.term + pd.DateOffset(months = 3)
fw1['before_4'] = fw1.term + pd.DateOffset(months = 4)
fw1['before_5'] = fw1.term + pd.DateOffset(months = 5)

In [ ]:
new = pd.merge(fw1_b3, fw1_b4, left_on = ['emd_nm','before_3'], right_on = ['emd_nm','before_4'])
new = pd.merge(new, fw1_b5, left_on = ['emd_nm','before_3'], right_on = ['emd_nm','before_5'])

In [ ]:
new.drop(['before_4', 'before_5'], axis = 1, inplace = True)

In [ ]:
new = new.rename(columns = {'before_b3' : 'term'})

In [ ]:
new['em_g_gap'] = new[['em_g_b5', 'em_g_b4', 'em_g_b3']].apply(max, axis=1) - new[['em_g_b5', 'em_g_b4', 'em_g_b3']].apply(min, axis=1)
new['em_g_gap_rate'] = new['em_g_b3'] / new['em_g_b5']
new['em_g_avg'] = (new.em_g_b5 + new.em_g_b4 + new.em_g_b3) // 3

In [ ]:
new['em_g_std'] = round(new[['em_g_b5', 'em_g_b4', 'em_g_b3']].apply(np.std, axis=1), 2)

In [ ]:
new.drop(['em_g_b3', 'em_g_b4', 'em_g_b5'], axis = 1, inplace = True)

In [ ]:
## 음식물쓰레기 변동량 데이터 저장
new.to_csv('../data/가공데이터/fw_data.csv')

# 내국인 유동인구 데이터 가공

In [ ]:
korean = korean.drop(['emd_cd','resd','city'],axis=1)
sex = ['여성', '남성']
age = [10,20,30,40,50,60]

for i,j in itertools.product(sex, age):
    if i == '여성' and j == 10:
        new_korean = korean[(korean['age'].isin([10,20])) & (korean['sex']=='여성')]
        new_korean = new_korean.groupby(['base_date','emd_nm']).sum().reset_index().drop(['age','time'],axis=1)
        new_korean.rename(columns={'resd_pop_cnt':'f1020_resd_pop_cnt', 'visit_pop_cnt':'f1020_visit_pop_cnt', 'work_pop_cnt':'f1020_work_pop_cnt'},inplace=True)
    elif j == 60:
        new = korean[(korean['age'].isin([60, 70, 80])) & (korean['sex'] == i)]
        if i == '여성':
            i = 'f'
            new = new.groupby(['base_date','emd_nm']).sum().reset_index().drop(['age','time','base_date', 'emd_nm'],axis=1)
            new.rename(columns={'resd_pop_cnt':f'{i}{j}_resd_pop_cnt', 'visit_pop_cnt':f'{i}{j}_visit_pop_cnt', 'work_pop_cnt':f'{i}{j}_work_pop_cnt'},inplace=True)
            new_korean = pd.concat([new_korean, new], axis = 1)
        else:
            i = 'm'
            new = new.groupby(['base_date','emd_nm']).sum().reset_index().drop(['age','time','base_date', 'emd_nm'],axis=1)
            new.rename(columns={'resd_pop_cnt':f'{i}{j}_resd_pop_cnt', 'visit_pop_cnt':f'{i}{j}_visit_pop_cnt', 'work_pop_cnt':f'{i}{j}_work_pop_cnt'},inplace=True)
            new_korean = pd.concat([new_korean, new], axis = 1)
    elif j == 30:
        new = korean[(korean['age'].isin([30, 40, 50])) & (korean['sex']==i)]
        if i == '여성':
            i = 'f'
            new = new.groupby(['base_date','emd_nm']).sum().reset_index().drop(['age','time','base_date', 'emd_nm'],axis=1)
            new.rename(columns={'resd_pop_cnt':f'{i}{j}_resd_pop_cnt', 'visit_pop_cnt':f'{i}{j}_visit_pop_cnt', 'work_pop_cnt':f'{i}{j}_work_pop_cnt'},inplace=True)
            new_korean = pd.concat([new_korean, new], axis = 1)
        else:
            i = 'm'
            new = new.groupby(['base_date','emd_nm']).sum().reset_index().drop(['age','time','base_date', 'emd_nm'],axis=1)
            new.rename(columns={'resd_pop_cnt':f'{i}{j}_resd_pop_cnt', 'visit_pop_cnt':f'{i}{j}_visit_pop_cnt', 'work_pop_cnt':f'{i}{j}_work_pop_cnt'},inplace=True)
            new_korean = pd.concat([new_korean, new], axis = 1)

    elif i == '남성' and j == 10:
        new = korean[(korean['age'].isin([60, 70, 80])) & (korean['sex'] == i)]
        i = 'm'
        new = new.groupby(['base_date','emd_nm']).sum().reset_index().drop(['age','time','base_date', 'emd_nm'],axis=1)
        new.rename(columns={'resd_pop_cnt':f'{i}{j}_resd_pop_cnt', 'visit_pop_cnt':f'{i}{j}_visit_pop_cnt', 'work_pop_cnt':f'{i}{j}_work_pop_cnt'},inplace=True)
        new_korean = pd.concat([new_korean, new], axis = 1)

In [ ]:
sex = ['f', 'm']
age = ['1020', '3050', '60up']
kind = ['resd', 'work', 'visit']

In [ ]:
cols = ['base_date', 'emd_nm']
for i, j in itertools.product(sex, age):
    col1 = f'{i}{j}_resd_pop_cnt'
    col2 = f'{i}{j}_work_pop_cnt'
    col3 = f'{i}{j}_visit_pop_cnt'
    cols.extend([col1,col2,col3])

In [ ]:
new_korean.columns = cols

In [ ]:
new_korean['work_pop_cnt'] = new_korean.f1020_work_pop_cnt + new_korean.f3050_work_pop_cnt + new_korean.f60up_work_pop_cnt + new_korean.m1020_work_pop_cnt + new_korean.m3050_work_pop_cnt + new_korean.m60up_work_pop_cnt
new_korean['visit_pop_cnt'] = new_korean.f1020_visit_pop_cnt + new_korean.f3050_visit_pop_cnt + new_korean.f60up_visit_pop_cnt + new_korean.m1020_visit_pop_cnt + new_korean.m3050_visit_pop_cnt + new_korean.m60up_visit_pop_cnt
new_korean['resd_pop_cnt'] = new_korean.f1020_resd_pop_cnt + new_korean.f3050_resd_pop_cnt + new_korean.f60up_resd_pop_cnt + new_korean.m1020_resd_pop_cnt + new_korean.m3050_resd_pop_cnt + new_korean.m60up_resd_pop_cnt
new_korean['floating_pop_cnt'] = new_korean['work_pop_cnt'] + new_korean['resd_pop_cnt']+new_korean['visit_pop_cnt']

In [ ]:
## 1020 비율
new_korean['floating_visit_rate_1020'] = (new_korean['f1020_visit_pop_cnt'] + new_korean['m1020_visit_pop_cnt'])/ new_korean['visit_pop_cnt']
new_korean['floating_resd_rate_1020'] = (new_korean['f1020_resd_pop_cnt'] + new_korean['m1020_resd_pop_cnt'])/ new_korean['resd_pop_cnt']

## 60up 비율
new_korean['floating_visit_rate_60up'] = (new_korean['f60up_visit_pop_cnt'] + new_korean['m60up_visit_pop_cnt'])/ new_korean['visit_pop_cnt']
new_korean['floating_resd_rate_60up'] = (new_korean['f60up_resd_pop_cnt'] +new_korean['m60up_resd_pop_cnt']) / new_korean['resd_pop_cnt']

## 여성 비율
new_korean['floating_f_visit_rate'] = (new_korean.f1020_visit_pop_cnt + new_korean.f3050_visit_pop_cnt + new_korean.f60up_visit_pop_cnt)/ new_korean['visit_pop_cnt']
new_korean['floating_f_resd_rate'] = (new_korean.f1020_resd_pop_cnt + new_korean.f3050_resd_pop_cnt + new_korean.f60up_resd_pop_cnt ) / new_korean['resd_pop_cnt']

In [ ]:
new_korean1 = new_korean.copy()
new_korean1['year'] = new_korean1.base_date.dt.year
new_korean1['month'] = new_korean1.base_date.dt.month

In [ ]:
new_korean1 = new_korean1.groupby(['year', 'month', 'emd_nm']).sum().reset_index()

In [ ]:
new_korean1['year'] = new_korean1['year'].astype(str)
new_korean1['month'] = new_korean1['month'].astype(str)
new_korean1['term'] = new_korean1['year'] + '-0' + new_korean1['month'] if len(new_korean1['month']) == 1 else new_korean1['year'] +'-'+ new_korean1['month']
new_korean1.drop(['year', 'month'], axis = 1, inplace = True)
new_korean1['term'] = new_korean1.term.apply(lambda x: x[:5] + '0' + x[-1] if len(x) == 6 else x) 

new_korean1['term'] = new_korean1['term'].apply(lambda x : dt.datetime.strptime(x, '%Y-%m'))

In [ ]:
new_korean1.drop([ 'f1020_resd_pop_cnt', 'f1020_work_pop_cnt',
       'f1020_visit_pop_cnt', 'f3050_resd_pop_cnt', 'f3050_work_pop_cnt',
       'f3050_visit_pop_cnt', 'f60up_resd_pop_cnt', 'f60up_work_pop_cnt',
       'f60up_visit_pop_cnt', 'm1020_resd_pop_cnt', 'm1020_work_pop_cnt',
       'm1020_visit_pop_cnt', 'm3050_resd_pop_cnt', 'm3050_work_pop_cnt',
       'm3050_visit_pop_cnt', 'm60up_resd_pop_cnt', 'm60up_work_pop_cnt',
       'm60up_visit_pop_cnt', 'work_pop_cnt', 'visit_pop_cnt', 'resd_pop_cnt'], axis = 1, inplace = True)

In [ ]:
new_korean1['before_3'] = new_korean1.term + pd.DateOffset(months = 3)
new_korean1['before_4'] = new_korean1.term + pd.DateOffset(months = 4)
new_korean1['before_5'] = new_korean1.term + pd.DateOffset(months = 5)

In [ ]:
new_korean1_b3 = new_korean1.drop(['term', 'before_4', 'before_5'], axis = 1)
new_korean1_b3.columns = [i + '_b3' for i in new_korean1_b3.columns]

new_korean1_b4 = new_korean1.drop(['term', 'before_3', 'before_5'], axis = 1)
new_korean1_b4.columns = [i + '_b4' for i in new_korean1_b4.columns]

new_korean1_b5 = new_korean1.drop(['term', 'before_4', 'before_3'], axis = 1)
new_korean1_b5.columns = [i + '_b5' for i in new_korean1_b5.columns]

new_korean1 = new_korean1.drop(['before_3', 'before_4', 'before_5'], axis = 1)

In [ ]:
new = pd.merge(new_korean1_b3, new_korean1_b4, left_on = ['emd_nm_b3','before_3_b3'], right_on = ['emd_nm_b4','before_4_b4'])
new = pd.merge(new, new_korean1_b5, left_on = ['emd_nm_b3','before_3_b3'], right_on = ['emd_nm_b5','before_5_b5'])

In [ ]:
new.drop(['before_4_b4', 'before_5_b5'], axis = 1, inplace = True)
new.drop(['emd_nm_b4','emd_nm_b5'], axis = 1, inplace = True)
new = new.rename(columns = {'before_3_b3' : 'term', 'emd_nm_b3' : 'emd_nm'})

In [ ]:
for i in new_korean1.columns[1:-1]:
    new[f'{i}_gap'] =new[[f'{i}_b5', f'{i}_b4', f'{i}_b3']].apply(max, axis=1) - new[[f'{i}_b5', f'{i}_b4', f'{i}_b3']].apply(min, axis=1)
    new[f'{i}_gap_rate'] =new[f'{i}_b3'] / new[f'{i}_b5']
    new[f'{i}_avg'] = (new[f'{i}_b5'] + new[f'{i}_b4'] + new[f'{i}_b3']) / 3
    new[f'{i}_std'] = round(new[[f'{i}_b5', f'{i}_b4', f'{i}_b3']].apply(np.std, axis=1), 2)

In [ ]:
for i in new_korean1.columns[1:-1]:
    new.drop([f'{i}_b5', f'{i}_b4', f'{i}_b3'], axis = 1, inplace = True)

In [ ]:
## 내국인 유동인구 데이터 저장
new.to_csv('../data/가공데이터/korean_floating_data.csv')

# 거주인구 데이터 가공

In [ ]:
f_resd = residence[residence.sex == '여성']
f_resd = f_resd.rename(columns = {'resid_reg_pop' : 'f_resid_reg_pop', 'foreign_pop' : 'f_foreign_pop', 'total_pop' : 'f_total_pop'})

In [ ]:
residence['base_year'] = residence['base_year'].astype(str)
residence['base_month'] = residence['base_month'].astype(str)

residence['term'] = residence['base_year'] + '-0' + residence['base_month'] if len(residence['base_month']) == 1 else residence['base_year'] +'-'+ residence['base_month']
residence.drop(['base_year', 'base_month'], axis = 1, inplace = True)
residence['term'] = residence.term.apply(lambda x: x[:5] + '0' + x[-1] if len(x) == 6 else x) 

residence['term'] = residence['term'].apply(lambda x : dt.datetime.strptime(x, '%Y-%m'))

In [ ]:
f_resd['base_year'] = f_resd['base_year'].astype(str)
f_resd['base_month'] = f_resd['base_month'].astype(str)

f_resd['term'] = f_resd['base_year'] + '-0' + f_resd['base_month'] if len(f_resd['base_month']) == 1 else f_resd['base_year'] +'-'+ f_resd['base_month']
f_resd.drop(['base_year', 'base_month'], axis = 1, inplace = True)
f_resd['term'] = f_resd.term.apply(lambda x: x[:5] + '0' + x[-1] if len(x) == 6 else x) 

f_resd['term'] = f_resd['term'].apply(lambda x : dt.datetime.strptime(x, '%Y-%m'))

In [ ]:
residence = residence.groupby(['term','emd_nm']).sum().reset_index().drop(['emd_cd'], axis = 1)
f_resd = f_resd.groupby(['term','emd_nm']).sum().reset_index().drop(['term','emd_nm','emd_cd'], axis = 1)

In [ ]:
residence = pd.concat([residence, f_resd], axis = 1)

In [ ]:
residence['f_resid_reg_rate'] =  residence['f_resid_reg_pop'] /residence['resid_reg_pop'] 
residence['f_foreign_rate'] =  residence['f_foreign_pop'] /residence['foreign_pop'] 
residence['f_total_rate'] =  residence['f_total_pop'] /residence['total_pop'] 

residence.drop(['f_resid_reg_pop', 'f_foreign_pop', 'f_total_pop'], axis = 1, inplace = True)

In [ ]:
# 유동인구 / 거주인구 비율 구하기
floating_resd_rate = new_korean1.floating_pop_cnt / residence.total_pop

In [ ]:
residence = pd.concat([residence, floating_resd_rate], axis =1).rename(columns = {0:'floating_resd_rate'})

In [ ]:
residence['before_3'] = residence.term + pd.DateOffset(months = 3)
residence['before_4'] = residence.term + pd.DateOffset(months = 4)
residence['before_5'] = residence.term + pd.DateOffset(months = 5)

In [ ]:
residence_b3 = residence.drop(['term', 'before_4', 'before_5'], axis = 1)
residence_b3.columns = [i + '_b3' for i in residence_b3.columns]

residence_b4 = residence.drop(['term', 'before_3', 'before_5'], axis = 1)
residence_b4.columns = [i + '_b4' for i in residence_b4.columns]

residence_b5 = residence.drop(['term', 'before_4', 'before_3'], axis = 1)
residence_b5.columns = [i + '_b5' for i in residence_b5.columns]

residence = residence.drop(['before_3', 'before_4', 'before_5'], axis = 1)

In [ ]:
new = pd.merge(residence_b3, residence_b4, left_on = ['emd_nm_b3','before_3_b3'], right_on = ['emd_nm_b4','before_4_b4'])
new = pd.merge(new, residence_b5, left_on = ['emd_nm_b3','before_3_b3'], right_on = ['emd_nm_b5','before_5_b5'])

In [ ]:
new.drop(['before_4_b4', 'before_5_b5'], axis = 1, inplace = True)
new.drop(['emd_nm_b4','emd_nm_b5'], axis = 1, inplace = True)
new = new.rename(columns = {'before_3_b3' : 'term', 'emd_nm_b3' : 'emd_nm'})

In [ ]:
for i in residence.columns[2:]:
    new[f'{i}_gap'] =new[[f'{i}_b5', f'{i}_b4', f'{i}_b3']].apply(max, axis=1) - new[[f'{i}_b5', f'{i}_b4', f'{i}_b3']].apply(min, axis=1)
    new[f'{i}_gap_rate'] =new[f'{i}_b3'] / new[f'{i}_b5']
    new[f'{i}_avg'] = (new[f'{i}_b5'] + new[f'{i}_b4'] + new[f'{i}_b3']) / 3
    new[f'{i}_std'] = round(new[[f'{i}_b5', f'{i}_b4', f'{i}_b3']].apply(np.std, axis=1), 2)

In [ ]:
for i in residence.columns[2:]:
    new.drop([f'{i}_b5', f'{i}_b4', f'{i}_b3'], axis = 1, inplace = True)

In [ ]:
# 선형종속 방지 위해
new.drop(['foreign_pop_avg', 'foreign_pop_gap'], axis = 1, inplace = True)

In [ ]:
## 거주인구 데이터 저장
new.to_csv('../data/가공데이터/resd_data_with_floating_rate.csv')

# 장기체류 외국인 유동인구 데이터 가공

In [ ]:
# LF_FRGN 데이터 가공

lt_frgn = lt_frgn[['base_date', 'time', 'nationality', 'city', 'emd_nm', 'resd_pop_cnt', 'work_pop_cnt', 'visit_pop_cnt','resd']]
lt_frgn.rename(columns ={'resd_pop_cnt' : 'lf_resd_pop_cnt', 'work_pop_cnt':'lf_work_pop_cnt', 'visit_pop_cnt':'lf_visit_pop_cnt'}, inplace = True)

lt_frgn_jeju = lt_frgn[lt_frgn.resd =='제주']
lt_frgn_notjeju = lt_frgn[lt_frgn.resd =='그외']
lt_frgn_notjeju.rename(columns = {'lf_work_pop_cnt':'nj_lf_work_pop_cnt', 'lf_visit_pop_cnt':'nj_lf_visit_pop_cnt'}, inplace = True)

lt_frgn = pd.merge(lt_frgn_jeju, lt_frgn_notjeju, on=['base_date','time','nationality','city','emd_nm'], how = 'outer')
lt_frgn = lt_frgn.fillna(0).drop(['resd_x', 'resd_y','lf_resd_pop_cnt_y'], axis = 1).rename(columns = {'lf_resd_pop_cnt_x':'lf_resd_pop_cnt'})

# 필요한 국적에 대한 데이터만 남기기
new_lf_data = lt_frgn[lt_frgn.nationality.isin(['JPN', 'CHN', 'TWN', 'MYS', 'VNM', 'THA', 'USA'])].groupby(['base_date', 'emd_nm','nationality']).sum().drop(['time'], axis = 1)
new_lf_data = new_lf_data.unstack().reset_index().fillna(0)

# 컬럼명 변경
col_name = list(itertools.product(['lf_resd_pop_cnt', 'lf_work_pop_cnt', 'lf_visit_pop_cnt', 'nj_lf_work_pop_cnt','nj_lf_visit_pop_cnt'], ['CHN', 'JPN', 'MYS', 'THA', 'TWN', 'USA', 'VNM']))
col_list = [i+'_'+j for i, j in col_name]
col_list = ['base_date', 'emd_nm'] + col_list
new_lf_data.columns = col_list


In [ ]:
#  ETC 컬럼 제작
new_lf_data['lf_resd_pop_cnt_ETC'] = new_lf_data['lf_resd_pop_cnt_MYS'] + new_lf_data['lf_resd_pop_cnt_THA'] + new_lf_data['lf_resd_pop_cnt_TWN'] + new_lf_data['lf_resd_pop_cnt_JPN']
new_lf_data['lf_work_pop_cnt_ETC'] = new_lf_data['lf_work_pop_cnt_MYS'] + new_lf_data['lf_work_pop_cnt_THA'] + new_lf_data['lf_work_pop_cnt_TWN'] + new_lf_data['lf_work_pop_cnt_JPN']
new_lf_data['lf_visit_pop_cnt_ETC'] = new_lf_data['lf_visit_pop_cnt_MYS'] + new_lf_data['lf_visit_pop_cnt_THA'] + new_lf_data['lf_visit_pop_cnt_TWN'] + new_lf_data['lf_visit_pop_cnt_JPN']

new_lf_data['nj_lf_work_pop_cnt_ETC'] = new_lf_data['nj_lf_work_pop_cnt_MYS'] + new_lf_data['nj_lf_work_pop_cnt_THA'] + new_lf_data['nj_lf_work_pop_cnt_TWN'] + new_lf_data['nj_lf_work_pop_cnt_JPN']
new_lf_data['nj_lf_visit_pop_cnt_ETC'] = new_lf_data['nj_lf_visit_pop_cnt_MYS'] + new_lf_data['nj_lf_visit_pop_cnt_THA'] + new_lf_data['nj_lf_visit_pop_cnt_TWN'] + new_lf_data['nj_lf_visit_pop_cnt_JPN']

new_lf_data = new_lf_data.drop(['lf_resd_pop_cnt_MYS', 'lf_resd_pop_cnt_THA', 'lf_resd_pop_cnt_TWN','lf_resd_pop_cnt_JPN',
             'lf_work_pop_cnt_MYS', 'lf_work_pop_cnt_THA', 'lf_work_pop_cnt_TWN','lf_work_pop_cnt_JPN',
             'lf_visit_pop_cnt_MYS', 'lf_visit_pop_cnt_THA', 'lf_visit_pop_cnt_TWN','lf_visit_pop_cnt_JPN',
             'nj_lf_work_pop_cnt_MYS', 'nj_lf_work_pop_cnt_THA', 'nj_lf_work_pop_cnt_TWN','nj_lf_work_pop_cnt_JPN',
             'nj_lf_visit_pop_cnt_MYS', 'nj_lf_visit_pop_cnt_THA', 'nj_lf_visit_pop_cnt_TWN','nj_lf_visit_pop_cnt_JPN'], axis = 1)
new_lf_data = new_lf_data.fillna(0)

In [ ]:
new_lf_data['rewo_pop_cnt_CVU_lf'] = new_lf_data.lf_work_pop_cnt_CHN + new_lf_data.nj_lf_work_pop_cnt_CHN + new_lf_data.lf_resd_pop_cnt_CHN +new_lf_data.lf_work_pop_cnt_USA + new_lf_data.nj_lf_work_pop_cnt_USA + new_lf_data.lf_resd_pop_cnt_USA +new_lf_data.lf_work_pop_cnt_VNM + new_lf_data.nj_lf_work_pop_cnt_VNM + new_lf_data.lf_resd_pop_cnt_VNM 

In [ ]:
new_lf_data['rewo_pop_cnt_ETC_lf'] =  new_lf_data.lf_work_pop_cnt_ETC + new_lf_data.nj_lf_work_pop_cnt_ETC + new_lf_data.lf_resd_pop_cnt_ETC 

In [ ]:
new_lf_data['visit_pop_cnt_CVU_lf'] = new_lf_data.lf_visit_pop_cnt_CHN + new_lf_data.nj_lf_visit_pop_cnt_CHN + new_lf_data.lf_visit_pop_cnt_USA + new_lf_data.nj_lf_visit_pop_cnt_USA+new_lf_data.lf_visit_pop_cnt_VNM + new_lf_data.nj_lf_visit_pop_cnt_VNM
new_lf_data['visit_pop_cnt_ETC_lf'] = new_lf_data.lf_visit_pop_cnt_ETC + new_lf_data.nj_lf_visit_pop_cnt_ETC 

In [ ]:
new_lf_data['floating_pop_cnt_CVU_lf'] = new_lf_data['rewo_pop_cnt_CVU_lf'] + new_lf_data['visit_pop_cnt_CVU_lf']
new_lf_data['floating_pop_cnt_ETC_lf'] = new_lf_data['rewo_pop_cnt_ETC_lf'] +new_lf_data['visit_pop_cnt_ETC_lf']

In [ ]:
new_lf_data['floating_pop_cnt_lf'] = new_lf_data['floating_pop_cnt_CVU_lf'] + new_lf_data['floating_pop_cnt_ETC_lf']

In [ ]:
new_lf_data['visit_rate_CVU_lf'] = new_lf_data.visit_pop_cnt_CVU_lf / new_lf_data.floating_pop_cnt_CVU_lf
new_lf_data['visit_rate_ETC_lf'] = new_lf_data.visit_pop_cnt_ETC_lf / new_lf_data.floating_pop_cnt_ETC_lf

In [ ]:
new_lf_data.drop(['lf_resd_pop_cnt_CHN', 'lf_resd_pop_cnt_VNM',
       'lf_resd_pop_cnt_USA', 'lf_work_pop_cnt_CHN', 'lf_work_pop_cnt_VNM',
       'lf_work_pop_cnt_USA', 'lf_visit_pop_cnt_CHN', 'lf_visit_pop_cnt_VNM',
       'lf_visit_pop_cnt_USA', 'nj_lf_work_pop_cnt_CHN',
       'nj_lf_work_pop_cnt_VNM', 'nj_lf_work_pop_cnt_USA',
       'nj_lf_visit_pop_cnt_CHN', 'nj_lf_visit_pop_cnt_VNM',
       'nj_lf_visit_pop_cnt_USA', 'lf_resd_pop_cnt_ETC', 'lf_work_pop_cnt_ETC',
       'lf_visit_pop_cnt_ETC', 'nj_lf_work_pop_cnt_ETC',
       'nj_lf_visit_pop_cnt_ETC'], axis = 1, inplace = True)

In [ ]:
new_lf_data1 = new_lf_data.copy()

In [ ]:
new_lf_data1['base_date']=pd.to_datetime(new_lf_data1['base_date'], format = '%Y-%m-%d')

In [ ]:
new_lf_data1['year'] = new_lf_data1.base_date.dt.year
new_lf_data1['month'] = new_lf_data1.base_date.dt.month
new_lf_data1 = new_lf_data1.groupby(['year', 'month', 'emd_nm']).sum().reset_index()

In [ ]:
new_lf_data1['year'] = new_lf_data1['year'].astype(str)
new_lf_data1['month'] = new_lf_data1['month'].astype(str)
new_lf_data1['term'] = new_lf_data1['year'] + '-0' + new_lf_data1['month'] if len(new_lf_data1['month']) == 1 else new_lf_data1['year'] +'-'+ new_lf_data1['month']
new_lf_data1.drop(['year', 'month'], axis = 1, inplace = True)
new_lf_data1['term'] = new_lf_data1.term.apply(lambda x: x[:5] + '0' + x[-1] if len(x) == 6 else x) 

new_lf_data1['term'] = new_lf_data1['term'].apply(lambda x : dt.datetime.strptime(x, '%Y-%m'))

In [ ]:
new_lf_data1['before_3'] = new_lf_data1.term + pd.DateOffset(months = 3)
new_lf_data1['before_4'] = new_lf_data1.term + pd.DateOffset(months = 4)
new_lf_data1['before_5'] = new_lf_data1.term + pd.DateOffset(months = 5)

In [ ]:
new_lf_data1_b3 = new_lf_data1.drop(['term', 'before_4', 'before_5'], axis = 1)
new_lf_data1_b3.columns = [i + '_b3' for i in new_lf_data1_b3.columns]

new_lf_data1_b4 = new_lf_data1.drop(['term', 'before_3', 'before_5'], axis = 1)
new_lf_data1_b4.columns = [i + '_b4' for i in new_lf_data1_b4.columns]

new_lf_data1_b5 = new_lf_data1.drop(['term', 'before_4', 'before_3'], axis = 1)
new_lf_data1_b5.columns = [i + '_b5' for i in new_lf_data1_b5.columns]

new_lf_data1 = new_lf_data1.drop(['before_3', 'before_4', 'before_5'], axis = 1)

In [ ]:
new = pd.merge(new_lf_data1_b3, new_lf_data1_b4, left_on = ['emd_nm_b3','before_3_b3'], right_on = ['emd_nm_b4','before_4_b4'])
new = pd.merge(new, new_lf_data1_b5, left_on = ['emd_nm_b3','before_3_b3'], right_on = ['emd_nm_b5','before_5_b5'])

In [ ]:
new.drop([ 'before_4_b4', 'before_5_b5'], axis = 1, inplace = True)
new.drop(['emd_nm_b4','emd_nm_b5'], axis = 1, inplace = True)
new = new.rename(columns = {'before_3_b3':'term', 'emd_nm_b3':'emd_nm'})

In [ ]:
for i in new_lf_data1.columns[1:-1]:
    new[f'{i}_gap'] =new[[f'{i}_b5', f'{i}_b4', f'{i}_b3']].apply(max, axis=1) - new[[f'{i}_b5', f'{i}_b4', f'{i}_b3']].apply(min, axis=1)
    new[f'{i}_gap_rate'] =new[f'{i}_b3'] / new[f'{i}_b5']
    new[f'{i}_avg'] = (new[f'{i}_b5'] + new[f'{i}_b4'] + new[f'{i}_b3']) / 3
    new[f'{i}_std'] = round(new[[f'{i}_b5', f'{i}_b4', f'{i}_b3']].apply(np.std, axis=1), 2)

In [ ]:
for i in new_lf_data1.columns[1:-1]:
    new.drop([f'{i}_b5', f'{i}_b4', f'{i}_b3'], axis = 1, inplace = True)

In [ ]:
## 장기체류 외국인 유동인구 데이터 저장
new.to_csv('../data/가공데이터/lf_floating_data.csv')

# 단기체류 외국인 유동인구 데이터 가공

In [ ]:
st_frgn1 = st_frgn.groupby(['base_date','emd_nm']).sum().reset_index().drop(['time','emd_cd'],axis=1)

In [ ]:
st_frgn1.base_date = st_frgn1.base_date.astype('datetime64')

In [ ]:
st_frgn1['year'] = st_frgn1.base_date.dt.year
st_frgn1['month'] = st_frgn1.base_date.dt.month

st_frgn1['year'] = st_frgn1['year'].astype(str)
st_frgn1['month'] = st_frgn1['month'].astype(str)

In [ ]:
st_frgn1['term'] = st_frgn1['year'] + '-0' + st_frgn1['month'] if len(st_frgn1['month']) == 1 else st_frgn1['year'] +'-'+ st_frgn1['month']b

In [ ]:
st_frgn1.drop(['year', 'month'], axis = 1, inplace = True)
st_frgn1['term'] = st_frgn1.term.apply(lambda x: x[:5] + '0' + x[-1] if len(x) == 6 else x) 

st_frgn1['term'] = st_frgn1['term'].apply(lambda x : dt.datetime.strptime(x, '%Y-%m'))

st_frgn2 = st_frgn1.groupby(['term','emd_nm']).sum().reset_index()

In [ ]:
st_frgn2['before_3'] = st_frgn2.term + pd.DateOffset(months = 3)
st_frgn2['before_4'] = st_frgn2.term + pd.DateOffset(months = 4)
st_frgn2['before_5'] = st_frgn2.term + pd.DateOffset(months = 5)

In [ ]:
new = pd.merge(st_frgn2_b3, st_frgn2_b4, left_on = ['emd_nm_b3','before_3_b3'], right_on = ['emd_nm_b4','before_4_b4'])
new = pd.merge(new, st_frgn2_b5, left_on = ['emd_nm_b3','before_3_b3'], right_on = ['emd_nm_b5','before_5_b5'])

In [ ]:
new.drop(['before_4_b4', 'before_5_b5'], axis = 1, inplace = True)
new.drop(['emd_nm_b4','emd_nm_b5'], axis = 1, inplace = True)
new = new.rename(columns = {'before_3_b3':'term', 'emd_nm_b3':'emd_nm'})

In [ ]:
for i in st_frgn2.columns[2:]:
    new[f'{i}_gap'] =new[[f'{i}_b5', f'{i}_b4', f'{i}_b3']].apply(max, axis=1) - new[[f'{i}_b5', f'{i}_b4', f'{i}_b3']].apply(min, axis=1)
    new[f'{i}_gap_rate'] =new[f'{i}_b3'] / new[f'{i}_b5'] ##추가
    new[f'{i}_avg'] = (new[f'{i}_b5'] + new[f'{i}_b4'] + new[f'{i}_b3']) / 3
    new[f'{i}_std'] = round(new[[f'{i}_b5', f'{i}_b4', f'{i}_b3']].apply(np.std, axis=1), 2)

In [ ]:
new.drop(['visit_pop_cnt_b3','visit_pop_cnt_b4', 'visit_pop_cnt_b5'], axis=1, inplace=True)

In [ ]:
new.rename(columns = {'visit_pop_cnt':'visit_pop_cnt_sf', 'visit_pop_cnt_gap':'visit_pop_cnt_gap_sf',
       'visit_pop_cnt_gap_rate':'visit_pop_cnt_gap_rate_sf', 'visit_pop_cnt_avg':'visit_pop_cnt_avg_sf', 'visit_pop_cnt_std':'visit_pop_cnt_std_sf' }, inplace = True)

In [ ]:
new = pd.merge(st_frgn2,new, how='right')

In [ ]:
new.to_csv('../data/가공데이터/short_foreign_data.csv', index=False)

# 카드소비 데이터 가공

In [ ]:
card['year'] = card.base_date.dt.year
card['month'] = card.base_date.dt.month

In [ ]:
card1 = card.groupby(['year', 'month', 'emd_nm' ,'mct_cat_nm']).sum()

In [ ]:
card1 = card1.unstack()

In [ ]:
cols = [ 'mart_' + i if j == '마트/슈퍼마켓' else ('food_' + i if j == '식품' else ('agri_' + i if j == '농축수산물' else ('deli_' + i if j == '배달' else (j + '_' + i)))) for i, j in card1.columns ]

In [ ]:
card1.columns = cols

In [ ]:
card1 = card1.reset_index()

In [ ]:
card1['year'] = card1['year'].astype(str)
card1['month'] = card1['month'].astype(str)
card1['term'] = card1['year'] + '-0' + card1['month'] if len(card1['month']) == 1 else card1['year'] +'-'+ card1['month']
card1.drop(['year', 'month'], axis = 1, inplace = True)
card1['term'] = card1.term.apply(lambda x: x[:5] + '0' + x[-1] if len(x) == 6 else x) 

In [ ]:
card1['term'] = card1['term'].apply(lambda x : dt.datetime.strptime(x, '%Y-%m'))

In [ ]:
card1 = card1[['emd_nm', 'term', 'agri_use_cnt', 'mart_use_cnt', 'deli_use_cnt','food_use_cnt',\
            'agri_use_amt', 'mart_use_amt', 'deli_use_amt', 'food_use_amt']]

In [ ]:
card1.fillna(0, inplace = True)

In [ ]:
for i in list(card1.columns)[2:]:
    card1[i] = card1[i].astype(int)

In [ ]:
for i in ('agri', 'mart', 'deli', 'food'):
    card1[f'{i}_per_use_amt'] = card1[f'{i}_use_amt'] / card1[f'{i}_use_cnt']

In [ ]:
card1.drop(['agri_use_cnt', 'mart_use_cnt', 'deli_use_cnt', 'food_use_cnt'], axis = 1, inplace = True)

In [ ]:
card1['before_3'] = card1.term + pd.DateOffset(months = 3)
card1['before_4'] = card1.term + pd.DateOffset(months = 4)
card1['before_5'] = card1.term + pd.DateOffset(months = 5)

In [ ]:
card1_b3 = card1.drop(['term', 'before_4', 'before_5'], axis = 1)
card1_b3.columns = [i + '_b3' for i in card1_b3.columns]

card1_b4 = card1.drop(['term', 'before_3', 'before_5'], axis = 1)
card1_b4.columns = [i + '_b4' for i in card1_b4.columns]

card1_b5 = card1.drop(['term', 'before_4', 'before_3'], axis = 1)
card1_b5.columns = [i + '_b5' for i in card1_b5.columns]

card1 = card1.drop(['before_3', 'before_4', 'before_5'], axis = 1)

In [ ]:
new = pd.merge(card1_b3, card1_b4, left_on = ['emd_nm_b3','before_3_b3'], right_on = ['emd_nm_b4','before_4_b4'])
new = pd.merge(new, card1_b5, left_on = ['emd_nm_b3','before_3_b3'], right_on = ['emd_nm_b5','before_5_b5'])

In [ ]:
new.drop(['before_4_b4', 'before_5_b5'], axis = 1, inplace = True)
new.drop(['emd_nm_b4','emd_nm_b5'], axis = 1, inplace = True)

new = new.rename(columns = {'before_3_b3' : 'term', 'emd_nm_b3' : 'emd_nm'})

In [ ]:
for i in card1.columns[2:]:
    new[f'{i}_gap'] =new[[f'{i}_b5', f'{i}_b4', f'{i}_b3']].apply(max, axis=1) - new[[f'{i}_b5', f'{i}_b4', f'{i}_b3']].apply(min, axis=1)
    new[f'{i}_gap_rate'] =new[f'{i}_b3'] / new[f'{i}_b5']
    new[f'{i}_avg'] = (new[f'{i}_b5'] + new[f'{i}_b4'] + new[f'{i}_b3']) / 3
    new[f'{i}_std'] = round(new[[f'{i}_b5', f'{i}_b4', f'{i}_b3']].apply(np.std, axis=1), 2)

In [ ]:
for i in card1.columns[2:]:
    new.drop([f'{i}_b5', f'{i}_b4', f'{i}_b3'], axis = 1, inplace = True)

In [ ]:
## 카드소비 데이터 저장
new.to_csv('../data/가공데이터/card_spending_data.csv')

# 전체 데이터로 합치기

In [ ]:
# 데이터 불러오기

fw = pd.read_csv("../data/가공데이터/fw_data.csv", encoding = 'utf-8')
korean = pd.read_csv('../data/가공데이터/korean_floating_data.csv',encoding = 'utf-8')
sf = pd.read_csv('../data/가공데이터/short_foreign_data2.csv', encoding = 'utf-8')
lf = pd.read_csv("../data/가공데이터/lf_floating_data.csv", encoding = 'utf-8')
resd = pd.read_csv('../data/가공데이터/resd_data_with_floating_rate.csv', encoding = 'utf-8')

fw.drop('Unnamed: 0', axis=1, inplace = True)
korean.drop('Unnamed: 0', axis=1, inplace = True)
lf.drop('Unnamed: 0', axis=1, inplace = True)
card.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
final = pd.merge(sf, lf, on=['term', 'emd_nm'], how='outer')
final1 = pd.merge(final, card, how='outer')
resdko = pd.merge(resd,korean, how='outer')
final3 = pd.merge(resdko, final1, how='outer')

In [ ]:
w = pd.read_csv('../data/외부데이터/기온강수량_최종.csv', encoding='cp949')
w['날짜']=w['날짜']+'-01'
w.rename(columns = {'날짜':'term'}, inplace=True)

In [ ]:
dust = pd.read_csv('../data/외부데이터/dust_mean_final.csv', encoding = 'cp949')

def change(i):
    i=i.replace(' ','0')
    return i

dust['날짜']=dust['날짜'].map(lambda x: change(x))
dust['날짜']=dust['날짜']+'-01'
dust.rename(columns = {'날짜':'term'}, inplace=True)

In [ ]:
wd = pd.merge(w,dust, how='outer')

In [ ]:
final4 = pd.merge(final3, wd, how='outer')

In [ ]:
final5 = final4.iloc[:1760]
final5.to_csv('final_check.csv', index=False)

In [ ]:
ch = pd.read_csv('final_check.csv', encoding = 'utf-8')
y = pd.read_csv('../data/가공데이터/y_data.csv', encoding = 'utf-8')
y.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
final6 = pd.merge(fw, ch, on = ['emd_nm', 'term'])
finla6 = finla6.fillna(0)
final6 = final6.rename(columns = {'평균기온(°C)':'temperature', '일강수량(mm)':'rain', '평균':'fine_dust'})

In [ ]:
final7 = pd.merge(final6, y, on = ['emd_nm', 'term'])

In [ ]:
data = final7.copy()

In [ ]:
data['peak'] = 0
data = data.set_index('term')

In [ ]:
# 성수기 날짜 개수 데이터 생성

data.loc['2018-07-01']['peak'] = 9
data.loc['2018-08-01']['peak'] = 12
data.loc['2018-09-01']['peak'] = 6
data.loc['2018-10-01']['peak'] = 3
data.loc['2018-12-01']['peak'] = 10
data.loc['2019-01-01']['peak'] = 1
data.loc['2019-02-01']['peak'] = 7
data.loc['2019-05-01']['peak'] = 4
data.loc['2019-06-01']['peak'] = 3
data.loc['2019-07-01']['peak'] = 5
data.loc['2019-08-01']['peak'] = 24
data.loc['2019-09-01']['peak'] = 6
data.loc['2019-10-01']['peak'] = 7
data.loc['2019-12-01']['peak'] = 1

data.loc['2020-01-01']['peak'] = 7
data.loc['2020-02-01']['peak'] = 8
data.loc['2020-04-01']['peak'] = 1
data.loc['2020-05-01']['peak'] = 5
data.loc['2020-07-01']['peak'] = 7
data.loc['2020-08-01']['peak'] = 22
data.loc['2020-09-01']['peak'] = 2
data.loc['2020-10-01']['peak'] = 9
data.loc['2020-12-01']['peak'] = 4

data = data.reset_index()

In [ ]:
data.to_csv('../data/최종데이터/for_train_data.csv', index=False)

In [ ]:
# 평가데이터 제작 위한 
# 독립변수 만들기

final8 = final6['2021-07-01':'2021-08-01']
final8['peak'] = 0


final8.loc['2021-07-01']['peak'] = 8
final8.loc['2021-08-01']['peak'] = 21

final8 = final8.reset_index()

final8.to_csv('../data/최종데이터/for_test_data.csv')